<a href="https://colab.research.google.com/github/HEK-Research/Multitask-Deep-Learning-Affinity-Prediction/blob/Dje/Sci_Reports_(2021)_Data_Curation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tried using other variations of some lines of codes, some worked, some didn't. They're mostly at the bottom section of this notebook along with a question or 2.

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
import pandas as pd
import numpy as np

##### After I have mounted drive to Colab, I can access any files in my Google Drive by modify the file_path (see below). You can make a copy of the entire dataset folder to your own Google Drive and modify the path accordingly. 

In [63]:
file_path = '/content/drive/MyDrive/Project_4_MTDNN/ChEMBL Datasets/CHEMBL3371.csv'

In [64]:
# Read in the original ChEMBL dataset
df = pd.read_csv(file_path,   delimiter=';', skiprows=0, low_memory=False)


In [65]:
#Size of the dataset
print(df.shape)

(9363, 45)


In [66]:
df #or print(df)

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties
0,CHEMBL1783973,NaN,None,388.56,0,4.04,3{22},Cc1ccc2c(c1)c1c(n2CCc2ccccn2)CCN(C2CCN(C)CC2)C1,IC50,NaN,...,Serotonin 6 (5-HT6) receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL1782051,1,Scientific Literature,Bioorg Med Chem Lett,2009.0,CHEMBL3307715,NaN
1,CHEMBL72554,NaN,None,408.52,0,4.14,5b,COc1cccc2c(CCN(C)C)cn(S(=O)(=O)c3ccc4ccccc4c3)c12,Ki,'=',...,Serotonin 6 (5-HT6) receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL1137194,1,Scientific Literature,Bioorg Med Chem Lett,2004.0,None,NaN
2,CHEMBL180221,NaN,None,386.90,0,4.25,12b,CN1CC=C(c2cn(S(=O)(=O)c3ccccc3Cl)c3ccccc23)CC1,Ki,'=',...,Serotonin 6 (5-HT6) receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL1141885,1,Scientific Literature,Bioorg Med Chem Lett,2005.0,CHEMBL3308376,NaN
3,CHEMBL4438777,NaN,None,364.42,0,4.91,20; PKS-183,COc1cccc2[nH]cc(CCNCc3ccc(-c4ccc(F)cc4)o3)c12,Ki,'=',...,Serotonin 6 (5-HT6) receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL4312034,1,Scientific Literature,Eur J Med Chem,2020.0,CHEMBL3307715,NaN
4,CHEMBL39,SEROTONIN,3.0,176.22,0,1.37,5-HT(1),NCCc1c[nH]c2ccc(O)cc12,Ki,'=',...,Serotonin 6 (5-HT6) receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL1145819,1,Scientific Literature,J Med Chem,2003.0,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9358,CHEMBL4543737,Al11,None,410.84,0,3.44,EUB0000710,Cc1cnc(-c2cc(Cl)ccc2F)cc1-n1c(=O)n(CC(N)=O)c2c...,Inhibition,NaN,...,Serotonin 6 (5-HT6) receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL4507308,54,Donated Chemical Probes - SGC Frankfurt,NaN,2021.0,None,Compound concentration None 10.0 µM
9359,CHEMBL4780458,NaN,None,457.42,0,4.04,EUB0000540,O=C(Nc1cccnn1)N1CC(C/C=C/c2cncc(Oc3ccc(C(F)(F)...,Inhibition,NaN,...,Serotonin 6 (5-HT6) receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL4507297,54,Donated Chemical Probes - SGC Frankfurt,NaN,2021.0,None,Compound concentration None 10.0 µM
9360,CHEMBL4537673,BAY-985,None,553.59,1,4.38,EUB0000305,C[C@H](c1ccnc(Nc2nc3ccc(-c4cc(N(C)C)ncn4)cc3[n...,Inhibition,NaN,...,Serotonin 6 (5-HT6) receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL4507318,54,Donated Chemical Probes - SGC Frankfurt,NaN,2021.0,None,Compound concentration None 10.0 µM
9361,CHEMBL4846153,NaN,None,426.93,0,3.49,18,O=S(=O)(c1cccc(Cl)c1)n1ccc2c(N3CCNCC3)nc3ccccc...,Ki,'=',...,Serotonin 6 (5-HT6) receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL4842371,1,Scientific Literature,J Med Chem,2021.0,None,TIME = 1.0 hr


In [67]:
#List all columns 
df.columns

Index(['Molecule ChEMBL ID', 'Molecule Name', 'Molecule Max Phase',
       'Molecular Weight', '#RO5 Violations', 'AlogP', 'Compound Key',
       'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value',
       'Standard Units', 'pChEMBL Value', 'Data Validity Comment', 'Comment',
       'Uo Units', 'Ligand Efficiency BEI', 'Ligand Efficiency LE',
       'Ligand Efficiency LLE', 'Ligand Efficiency SEI', 'Potential Duplicate',
       'Assay ChEMBL ID', 'Assay Description', 'Assay Type', 'BAO Format ID',
       'BAO Label', 'Assay Organism', 'Assay Tissue ChEMBL ID',
       'Assay Tissue Name', 'Assay Cell Type', 'Assay Subcellular Fraction',
       'Assay Parameters', 'Assay Variant Accession', 'Assay Variant Mutation',
       'Target ChEMBL ID', 'Target Name', 'Target Organism', 'Target Type',
       'Document ChEMBL ID', 'Source ID', 'Source Description',
       'Document Journal', 'Document Year', 'Cell ChEMBL ID', 'Properties'],
      dtype='object')

In [68]:
#How to list specific columns
df.loc[:,['Molecular Weight', 'Standard Type', 'Standard Value','Standard Units','AlogP']]

,Molecular Weight,Standard Type,Standard Value,Standard Units,AlogP
0,388.56,IC50,NaN,NaN,4.04
1,408.52,Ki,5.00,nM,4.14
2,386.90,Ki,10.00,nM,4.25
3,364.42,Ki,132.00,nM,4.91
4,176.22,Ki,65.00,nM,1.37
...,...,...,...,...,...
9358,410.84,Inhibition,14.14,%,3.44
9359,457.42,Inhibition,7.76,%,4.04
9360,553.59,Inhibition,-4.38,%,4.38
9361,426.93,Ki,3.00,nM,3.49


###### Some simple analysis to check with the filters
1. Only compounds with reported direct interactions. This corresponds to 'Assay Type' = 'B' 

For example, this dataset includes assay type:
* 'F' - Biological effect of a compound
* 'B' - Binding of compounds to a molecular target, e.g. Ki, IC50, Kd
* 'A' - ADME data, e.g. t1/2, oral bioavailability

(For further information: https://chembl.gitbook.io/chembl-interface-documentation/frequently-asked-questions/chembl-data-questions) 

In [69]:
#https://www.askpython.com/python/built-in-methods/unique-values-from-a-dataframe below

In [70]:
#Used when data has more than a single dimension
#Finds unique values from a particular column
           #print(pd.DataFrame.columns-'AlogP'.unique())

In [71]:
#Used when data is of 1-Dimensional
#print(pd.DataFrame.unique())

In [72]:
#Similar to what I was attempting above
print(df['Assay Type'].unique())

['F' 'B' 'A']


2. Exact activity measures. This corresponds to 'Standard Relation' = "'='"

In [73]:
print(df['Standard Relation'].unique())

[nan "'='" "'<'" "'>'" "'~'" "'<='"]


3. Only standard potency measurements were considered. 

This is a bit complicated. However, we can utilize the correlation between different standard type (activity type) vs. pChEMBL Value. 

All numerical measurements reported in various units were standardized to 'Standard Value" in nM. This is converted to a negative logarithmic scale.

IC50 = 1 nm is equivalent to pChEMBL Value of 9. 
Higher the potency (lower IC50), smaller the pChEMBL value. (Think in terms of aciditiy vs pH) 

Which means, only standard potency measurements would have numerical pChEMBL Values reported. 

I did a groupby analysis to identify and extract 'Standard Type' with numerical 'pChEMBL Values'.

In [74]:
print(df['Standard Type'].unique())

['IC50' 'Ki' 'Activity' 'Imax' 'EC50' 'Inhibition' 'Emax' 'Kbapp' 'Kd'
 'Kb' 'Efficacy' '%max' 'Log Ki' 'IA' 'Delta pKi' 'pKb' 'INH' '% Ctrl'
 'pA2' 'Displacement' 'pKi'
 '% Inhibition of Control Agonist Response (Mean n=2)'
 'Activation (% of control)' 'effect'
 '% of Control Agonist Response (Mean n=2)' 'Affinity'
 'Mean fold stimulation'
 '% Inhibition of Control Specific Binding (Mean n=2)'
 'Inhibition (% of control)']


In [75]:
# group the dataframe by type and inspect the pchembl_value column
grouped = df.groupby('Standard Type')['pChEMBL Value']

activities = []
#Functions
# check if any group has only numerical values or only None values
for group_name, group_values in grouped:
    group_values = group_values.dropna()
    if len(group_values) == 0:
        print(f"All values in group {group_name} are None.")
    elif all(isinstance(val, float) for val in group_values):
        print(f"All values in group {group_name} are numerical.")
        activities.append(group_name)
    else:
        print(f"Group {group_name} has mixed data types.")
        
print("Standard potency measurement types to keep are:", activities)

All values in group % Ctrl are None.
All values in group % Inhibition of Control Agonist Response (Mean n=2) are None.
All values in group % Inhibition of Control Specific Binding (Mean n=2) are None.
All values in group % of Control Agonist Response (Mean n=2) are None.
All values in group %max are None.
All values in group Activation (% of control) are None.
All values in group Activity are None.
All values in group Affinity are None.
All values in group Delta pKi are None.
All values in group Displacement are None.
All values in group EC50 are numerical.
All values in group Efficacy are None.
All values in group Emax are None.
All values in group IA are None.
All values in group IC50 are numerical.
All values in group INH are None.
All values in group Imax are None.
All values in group Inhibition are None.
All values in group Inhibition (% of control) are None.
All values in group Kb are None.
All values in group Kbapp are None.
All values in group Kd are numerical.
All values in gr

4. All compounds annotated as ('inactive', 'not active', 'inconclusive', 'potential transcription error', or 'pan assay interference compounds (PAINS)') were discarded. 

I also did some investigation of how to best implement this filter, as explained below:

In [76]:
# The 'Comment' column contains a mixture of Non, string, and numerical values:
df.Comment[0:20]

0          Not Active
1                 NaN
2                 NaN
3                 NaN
4                 NaN
5              309762
6              309798
7                 NaN
8                 NaN
9                 NaN
10             221395
11                NaN
12             222882
13             222783
14                NaN
15                NaN
16    Partial agonist
17                NaN
18                NaN
19                NaN
Name: Comment, dtype: object

In [77]:
# I used regular expressions to replace all numerical values by NaN
               #https://www.bogotobogo.com/python/python_regularExpressions.php
#'r' is 'raw string' & tells Python that nothing in this string should be escaped

#Does "'^\d+$'" mean any character except those with one or more numeric digits?
#$ matches the end of a string.
#regex: special sequence that uses search patterns to find a (set of) string/ presence or absence of a text.

df['Comment'] = df['Comment'].replace(to_replace=r'^\d+$', value=np.nan, regex=True)
print(df.Comment[0:20])
print(df['Comment'].unique())

0          Not Active
1                 NaN
2                 NaN
3                 NaN
4                 NaN
5                 NaN
6                 NaN
7                 NaN
8                 NaN
9                 NaN
10                NaN
11                NaN
12                NaN
13                NaN
14                NaN
15                NaN
16    Partial agonist
17                NaN
18                NaN
19                NaN
Name: Comment, dtype: object
['Not Active' nan 'Partial agonist' 'Not Determined'
 'Not Active (inhibition < 50% @ 10 uM and thus dose-reponse curve not measured)'
 'Active' 'Agonist' 'Dose-dependent effect' 'Antagonist' 'Slightly Active'
 'Dose-Dependent Effect']


In [78]:
# I noticed that it appeared that any compound with numerical pChEMBL Value does not have Comment, or in another word, have only NaN. 
# The following groupby analysis is to confirm whether this observation is true:

grouped = df.groupby('Comment')['pChEMBL Value']

# check if any group has only numerical values or only None values
for group_name, group_values in grouped:
    group_values = group_values.dropna()
    if len(group_values) == 0:
        print(f"All values in group {group_name} are None.")
    elif all(isinstance(val, float) for val in group_values):
        print(f"All values in group {group_name} are numerical.")
        
    else:
        print(f"Group {group_name} has mixed data types.")

All values in group Active are None.
All values in group Agonist are None.
All values in group Antagonist are None.
All values in group Dose-Dependent Effect are None.
All values in group Dose-dependent effect are None.
All values in group Not Active are None.
All values in group Not Active (inhibition < 50% @ 10 uM and thus dose-reponse curve not measured) are None.
All values in group Not Determined are None.
All values in group Partial agonist are None.
All values in group Slightly Active are None.


We can conclude that keep only rows with 'Comment' = NaN, we can drop any compounds with undesirable annotations. 